## Zaawansowane Metody Inteligencji Obliczeniowej
# Zadanie domowe 1
### Prowadzący: Michał Kempka, Marek Wydmuch
### Autor: Michał Ciesielski 145325

## Wprowadzenie

Całe zadanie jest oparte o różne wersje środowiska `VacuumEnvironemnt`, które rozważaliśmy na zajęciach.
Środowisko zaimplementowane jest w bibliotece aima3 (https://github.com/ArtificialIntelligenceToolkit/aima3),
która zawiera kod do książki "Artificial Intelligence: A Modern Approach".

#### Uwaga: Możesz dowolnie modyfikować elementy tego notebooka (wstawiać komórki i zmieniać kod) o ile nie napisano gdzieś inaczej.

In [1]:
# Zainstaluj bibliotekę OpenAI Gym
!pip install aima3

     ---------------------------------------- 0.0/150.2 kB ? eta -:--:--
     ---------------------------- --------- 112.6/150.2 kB 2.2 MB/s eta 0:00:01
     -------------------------------------- 150.2/150.2 kB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/78.5 kB ? eta -:--:--
     ---------------------------------------- 78.5/78.5 kB ? eta 0:00:00
     ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
     ------- -------------------------------- 0.2/1.3 MB 7.6 MB/s eta 0:00:01
     ----------------------- ---------------- 0.8/1.3 MB 8.1 MB/s eta 0:00:01
     ---------------------------------------- 1.3/1.3 MB 9.3 MB/s eta 0:00:00


In [1]:
# Zaimportuj wszystkie jego elementy
from aima3.agents import *

Wszystkie używane przez nas elementy biblioteki są zaimplementowane w pliku: https://github.com/ArtificialIntelligenceToolkit/aima3/blob/master/aima3/agents.py

# Zad. 1 - Cechy środowiska odkurzacza (1 pkt.)

Wypisz cechy poniżej używanego środowiska zgodnie z klasyfikacją z wykładu 1.
Dla ciągłości/dyskretności określ cechy osobno w stosunku do czasu, akcji i przestrzeni stanów.
W razie wątpliwości uzasadnij swój wybór.

Odpowiedź: 
1. Obserwowalność środowiska - całkowicie obserwowalne
2. Liczność agentów w środowisku - jednoagentowe
3. Determinizm środowiska - deterministyczne
4. Statyczne środowisko
5. Ciągłość środowiska:
* czas - ciągłe (posunięcia odkurzacza mogą być nieskończone jeśli zostanie źle wyuczony lub nie będzie zawartego warunku stopu)
* akcje - dyskretne (skończona liczba możliwości "wykonania ruchu" w danym stanie)
* przestrzeń stanów - dyskretne (skończona liczba możliwych stanów dla danej charakterystyki pewnego środowiska)
6. Model środowiska - znany

Tip: Możesz sprawdź implementacje środowiska w pliku podanym powyżej, lub wywnioskować cechy na wykonując poniższe fragmenty kodu.

In [2]:
# Stwórz nowe środowisko świata odkurzacza
env = TrivialVacuumEnvironment()

In [4]:
# Sprawdź aktualny status środowiska
env.status

{(0, 0): 'Clean', (1, 0): 'Dirty'}

In [4]:
import collections
collections.Callable = collections.abc.Callable

In [9]:
# Utwórz agenta refleksyjnego
agent = ReflexVacuumAgent()
agent.is_alive()

True

In [10]:
# Dodaj agenta do środowiska. Owijamy go w TraceAgent'a, żeby zobaczyć co robi.
env.add_thing(TraceAgent(agent))

In [11]:
# Zobacz gdzie jest agent
for loc in [loc_A, loc_B]:
    print('loc {0}: {1}'.format(loc, env.list_things_at(loc)))
# Lub:
agent.location

loc (0, 0): [<Agent>]
loc (1, 0): []


(0, 0)

In [12]:
# Wykonaj 10 kroków
env.run(10)

<Agent> perceives ((0, 0), 'Clean') and does Right
<Agent> perceives ((1, 0), 'Dirty') and does Suck
<Agent> perceives ((1, 0), 'Clean') and does Left
<Agent> perceives ((0, 0), 'Clean') and does Right
<Agent> perceives ((1, 0), 'Clean') and does Left
<Agent> perceives ((0, 0), 'Clean') and does Right
<Agent> perceives ((1, 0), 'Clean') and does Left
<Agent> perceives ((0, 0), 'Clean') and does Right
<Agent> perceives ((1, 0), 'Clean') and does Left
<Agent> perceives ((0, 0), 'Clean') and does Right


In [13]:
# Sprawdź jak środowisko oceniło jakość agenta.
agent.performance

1

In [14]:
# Moglibyśmy ocenić oczekiwaną jakość agenta dokładniej..., ale tylko ją oszacujemy (1000 powtórzeń).
# Zakładamy, że symulacja trwa 50 kroków.

compare_agents(TrivialVacuumEnvironment, [ReflexVacuumAgent, ModelBasedVacuumAgent], 1000, 50)

[(<function aima3.agents.ReflexVacuumAgent()>, -38.89),
 (<function aima3.agents.ModelBasedVacuumAgent()>, 9.1)]

# Zad. 2 - Cechy zmodyfikowanego środowisko odkurzacza (1 pkt).

Wypisz cechy poniżej używanego środowiska zgodnie z klasyfikacją z wykładu 1.
Dla ciągłości/dyskretności określ cechy osobno w stosunku do czasu, akcji i przestrzeni stanów.
W razie wątpliwości uzasadnij swój wybór.

Odpowiedź: Miejsce na Twoje odpowiedzi

In [2]:
# Rozszerzmy implementacje TrivialVacuumEnvironment

import random

class TrivialVacuumEnvironmentWithCats(TrivialVacuumEnvironment):
    def __init__(self, random_dirt_prob=0.05, seed=None):
        super(TrivialVacuumEnvironmentWithCats, self).__init__()
        self.random = random.Random(seed)
        self.random_dirt_prob = random_dirt_prob

    def execute_action(self, agent, action):
        """Change agent's location and/or location's status; track performance; add dirt;
        Score 10 for each dirt cleaned; -1 for each move."""
        # Same as in case of TrivialVacuumEnvironment
        if action == 'Right':
            agent.location = loc_B
            agent.performance -= 1
        elif action == 'Left':
            agent.location = loc_A
            agent.performance -= 1
        elif action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                agent.performance += 10
            self.status[agent.location] = 'Clean'

        # Cats can make either location dirty
        for loc in [loc_A, loc_B]:
            if self.random.random() < self.random_dirt_prob:
                self.status[loc] = 'Dirty'

In [5]:
# Przetestujmy domyślnych agentów w nowym środowisku

def env_factory():
    return TrivialVacuumEnvironmentWithCats(random_dirt_prob=0.05)

compare_agents(env_factory, [ReflexVacuumAgent, ModelBasedVacuumAgent], 1000, 50)

[(<function aima3.agents.ReflexVacuumAgent()>, 11.842),
 (<function aima3.agents.ModelBasedVacuumAgent()>, 33.1)]

# Zad. 3 - Własny program agenta (8 pkt.)

Napisz program agenta, który będzie (średnio) dużo lepszy dla tego środowiska (50 kroków, z random_dirt_prob=0.05) niż ModelBaseVacuumAgent oraz ReflexVacuumAgent. Opisz działanie swojego programu, na podstawie jaki przesłanek on działa, jakbyś go zmodyfikował gdyby prawdopodobieństwo zabrudzenia pokoju (random_dirt_prob) się zmieniło?

Punktacja za wynik (sprawdzarka zrobi 50000 powtórzeń):
* \> 41: 1 pkt.
* \> 42: 2 pkt.
* \> 43: 3 pkt.
* \> 44: 4 pkt.
* \> 45: 5 pkt.
* \> 46: 6 pkt.

\+ 2 pkt. za opis.

#### Uwaga: nie zmieniaj nazwy klasy `MyVacuumAgent`. Nie dopisuj do komórki z klasą innego kodu. Możesz zdefiniować funkcje pomocnicze w tej samej komórce (sprawdzarka wyciągnie ze zgłoszonego notebooka wyłącznie komórkę z klasę o nazwie `MyVacuumAgent` do sprawdzenia).

In [32]:
# Klasa MyVacuumAgent wypełniona przykładowym kodem agenta z modelem

def MyVacuumAgent():
    import random
    model = {loc_A: None, loc_B: None}
    random = random.Random(None)
    random_dirt_prob = 0.13
    def program(percept):
        # Zwróć jedną z dostępnych akcji: {'Suck', 'Right', 'Left', 'NoOP'}
        nonlocal random_dirt_prob
        nonlocal random
        random_dirt_prob_2 = random_dirt_prob
        location, status = percept
        model[location] = status
        cats_made_room_dirty = (random.random() < random_dirt_prob)
        if status == 'Clean' and location == loc_A and cats_made_room_dirty:
            random_dirt_prob += 0.011
            return 'Right'
        if status == 'Clean' and location == loc_B and cats_made_room_dirty:
            random_dirt_prob += 0.011
            return 'Left'
        
        random_dirt_prob -= 0.0001
        
        if model[loc_A] == model[loc_B] == 'Clean':
            return 'NoOp'
        elif status == 'Dirty':
            return 'Suck'
        elif location == loc_A:
            return 'Right'
        elif location == loc_B:
            return 'Left'
        
    return Agent(program)

In [26]:
# Przetestuj swojego agenta
def env_factory():
    return TrivialVacuumEnvironmentWithCats(random_dirt_prob=0.05)

compare_agents(env_factory, [MyVacuumAgent], 50000, 50)

[(<function __main__.MyVacuumAgent()>, 45.54656)]

In [11]:
# Przetestuj swojego agenta
def env_factory():
    return TrivialVacuumEnvironmentWithCats(random_dirt_prob=0.05)

compare_agents(env_factory, [MyVacuumAgent], 50000, 50)

[(<function __main__.MyVacuumAgent()>, 45.6793)]

In [25]:
# Klasa MyVacuumAgent wypełniona przykładowym kodem agenta z modelem

def MyVacuumAgent():
    import random
    model = {loc_A: None, loc_B: None}
    random = random.Random(None)
    random_dirt_prob = 0.13
    
    step = 0
    i = 8
    
    def program(percept):
        # Zwróć jedną z dostępnych akcji: {'Suck', 'Right', 'Left', 'NoOP'}
        nonlocal random_dirt_prob
        nonlocal random
        nonlocal step, i
        random_dirt_prob_2 = random_dirt_prob
        location, status = percept
#         model[location] = status
#         cats_made_room_dirty = (random.random() < random_dirt_prob)
#         if status == 'Clean' and location == loc_A and cats_made_room_dirty:
#             random_dirt_prob += 0.011
#             return 'Right'
#         if status == 'Clean' and location == loc_B and cats_made_room_dirty:
#             random_dirt_prob += 0.011
#             return 'Left'
        
        step += 1
        cats_made_room_dirty = (random.random() < random_dirt_prob)
#         random_dirt_prob -= 0.0001
        
        if status == 'Dirty':
#             i = 10
            return 'Suck'
        elif step == 49:
#             step = 0
            if location == loc_A:
                return 'Right'
            return 'Left'
        elif location == loc_A and i <= 0:
            i = 8
            return 'Right'
        elif location == loc_B and i <= 0:
            i = 8
            return 'Left'
        elif status == 'Clean':
            if cats_made_room_dirty:
                i -= 1.3
            else:
                i -= 1
            return 'NoOp'
        
    return Agent(program)